In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from keras.layers import (Dropout, Input, Dense, Conv2D, 
                          MaxPooling2D, GlobalAveragePooling2D, 
                          UpSampling2D, Conv2DTranspose, 
                          Reshape, Flatten, Activation, 
                          BatchNormalization, Concatenate)
from keras.models import Model, Sequential
from keras.preprocessing import image
from keras.layers.advanced_activations import LeakyReLU
from keras.initializers import RandomNormal
# from keras.optimizers import adam_v2
import tensorflow as tf
import keras

import sys
import cv2
import os

In [2]:
DATA_DIR = 'English Alphabet/'

In [24]:
samples = []    #len=1900
label = []      #len=1900 
for i in range(1, 27):
 for filename in os.listdir(f'English Alphabet/{i}'):
        
    img = cv2.imread(os.path.join(f'English Alphabet/{i}', filename))     
    gray_img = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    resized_image = cv2.resize(gray_img, (32,32))
   #  img_flatten = resized_image.reshape(-1)
    samples.append(resized_image)
    label.append(i)

In [26]:
import cv2
# Normalize the image
normalized_sample=[]
for k in samples:
   img_normalized = cv2.normalize(k, None, 0, 1.0,
   cv2.NORM_MINMAX, dtype=cv2.CV_32F)
   normalized_sample.append(k)
   

In [27]:
(samples[2]).shape

(32, 32)

In [28]:
x = np.array(samples)
y = np.array(label).reshape(-1, 1)

In [29]:
randomize = np.arange(len(x))
np.random.shuffle(randomize)
x = x[randomize]
y = y[randomize]

In [30]:
y.shape

(520, 1)

In [31]:
# labels one hot
y = np.zeros((520, 26))         

for i in range(len(label)):
    y[i, label[i]-1] = 1    

In [35]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [36]:
import tensorflow as tf
base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(32,32, 3))

In [37]:
base_model.trainable = False
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

In [38]:
img_input = Input(shape=(32,32, 1))
img_conc = Concatenate()([img_input, img_input, img_input]) 

In [39]:
out_basemodel = base_model(img_conc)

In [49]:
out1 = Flatten()(out_basemodel)
out2 = Dense(256, activation='relu')(out1)
out3 = Dense(128, activation='relu')(out2)
out4 = Dense(64, activation='relu')(out3)
out5 = Dense(26, activation='softmax')(out4)

In [50]:
model = Model(img_input, out5)

In [51]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [52]:
# build the model
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
13/13 [==============================] - 6s 336ms/step - loss: 20.8982 - accuracy: 0.0240
Epoch 2/5
13/13 [==============================] - 4s 297ms/step - loss: 3.3024 - accuracy: 0.0673
Epoch 3/5
13/13 [==============================] - 4s 289ms/step - loss: 3.1653 - accuracy: 0.0865
Epoch 4/5
13/13 [==============================] - 4s 300ms/step - loss: 3.1435 - accuracy: 0.0938
Epoch 5/5
13/13 [==============================] - 4s 334ms/step - loss: 3.0910 - accuracy: 0.0962


In [53]:
pred_train= model.predict(x_train)
scores = model.evaluate(x_train, y_train, verbose=0)
print('Accuracy on training data: {}% \n Error on training data: {}'.format(scores[1], 1 - scores[1]))   
 
pred_test= model.predict(x_test)
scores2 = model.evaluate(x_test, y_test, verbose=0)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores2[1], 1 - scores2[1])) 

Accuracy on training data: 0.14182692766189575% 
 Error on training data: 0.8581730723381042
Accuracy on test data: 0.08653846383094788% 
 Error on test data: 0.9134615361690521
